 # 5.2 Setting up a network model and starting a first training

In this exercise, we are going to practise, how to set up a neural network model and perform a first training with this network.
We will use the DermaMNIST from the MedMNIST datasets, which you have seen last lecture.

In [ ]:
!git clone https://github.com/MedMNIST/MedMNIST.git

In [ ]:
# Imports

!pip install --upgrade -q gspread
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
from datetime import datetime
import pytz
import datetime
import random

tz = pytz.timezone('Europe/Berlin')
gc = gspread.authorize(GoogleCredentials.get_application_default())

import os
import sys
# from tqdm import trange
# from tqdm import tqdm
# from skimage.util import montage
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision as torchvision

import MedMNIST.medmnist as medmnist
from MedMNIST.medmnist.dataset import PathMNIST, ChestMNIST, DermaMNIST, OCTMNIST, PneumoniaMNIST, RetinaMNIST, BreastMNIST, OrganMNISTAxial, OrganMNISTCoronal, OrganMNISTSagittal
from MedMNIST.medmnist.evaluator import getAUC, getACC
from MedMNIST.medmnist.info import INFO

In [ ]:
print("Version:", medmnist.__version__)

In [ ]:
# various MedMNIST datasets

data_flag = 'dermamnist'
download = True
input_root = 'tmp_data/'

flag_to_class = {
    "pathmnist": PathMNIST,
    "chestmnist": ChestMNIST,
    "dermamnist": DermaMNIST,
    "octmnist": OCTMNIST,
    "pneumoniamnist": PneumoniaMNIST,
    "retinamnist": RetinaMNIST,
    "breastmnist": BreastMNIST,
    "organmnist_axial": OrganMNISTAxial,
    "organmnist_coronal": OrganMNISTCoronal,
    "organmnist_sagittal": OrganMNISTSagittal,
}

DataClass = flag_to_class[data_flag]

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])
label_dict = info['label']

print(f"Info:\n{info}\n")
print(f"Task:\n{task}\n")
print(f"Channels:\n{n_channels}\n")
print(f"Number of classes:\n{n_classes}\n")
print(f"Label:\n{label_dict}\n")

The first time you run this there is probably an error. Just click on "RUNTIME" --> "RESTART RUNTIME" and run this cell again.

In [ ]:
# @title Result Form
gsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1oNM74xlAKuSJESKm2ggO5sduKYdVab4xzTShiG0lB34/edit?usp=sharing")

def write_result(task_number, result=None):
  worksheet = gsheet.worksheet("task{}".format(task_number))
  current_time = datetime.datetime.now(tz).strftime("%X")
  current_date = str(datetime.date.today())
  if result:
    worksheet.append_row([student_name, current_time, current_date, result])
    print("Task {} successfully solved by {} at {} with result: {}".format(task_number, student_name, current_time, result))
  else:
    worksheet.append_row([student_name, current_time, current_date])
    print("Task {} successfully solved by {} at {}".format(task_number, student_name, current_time))

print("Reporting enabled - write_result(number_of_task, result='your result') ")


Reporting enabled - write_result(number_of_task, result='your result') 


In [ ]:
student_name = "FlorianH"
assert student_name != "yourName"

## Defining the augmentations

As described in the last exercise, we now define the augmentations:

In [ ]:
# Imagenet values
norm_mean = (0.4914)
norm_std = (0.2023)

# define the transformaitons the images go through each time it is used for training
# includes augmentation AND normalization as described above
augmentation_train = transforms.Compose([
                                  # resize image to the network input size
                                  transforms.Resize((28,28)),
                                  # rotate the image with a certain angle range, randomly chosen
                                  transforms.RandomRotation(degrees=20),
                                  # convert the image into a tensor so it can be processed by the GPU
                                  transforms.ToTensor(),
                                  # normalize the image with the mean and std of ImageNet
                                  transforms.Normalize(norm_mean, norm_std),
                                   ])

In [ ]:
# no augmentation for the test data only resizing, conversion to tensor and normalization
augmentation_test = transforms.Compose([
                    transforms.Resize((28,28)),
                    transforms.ToTensor(),
                    transforms.Normalize(norm_mean, norm_std),
                    ])


We are again using a form to monitor the progress for the different tasks:

In [ ]:
# Confirm that you are ready to go:
write_result(0, 'Ready!!!')

set up datasets for training, validation and testing

In [ ]:
# load the data
train_dataset = DataClass(root=input_root, split='train', transform=augmentation_train, download=download)
test_dataset = DataClass(root=input_root, split='test', transform=augmentation_test, download=download)
val_dataset = DataClass(root=input_root, split='val', transform=augmentation_test, download=download)

In [ ]:
### length of training dataset
print(f"Length of training dataset: {len(train_dataset)}")

In [ ]:
### and some more detailed information about all splts
print("===================")
print(train_dataset)
print("===================")
print(val_dataset)
print("===================")
print(test_dataset)


create dataloaders for easy data handling, assigning according transforms and splits

In [ ]:
BATCH_SIZE = 128
### encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
### the next() function returns the next item from the iterator.
batch_images, batch_labels = next(iter(train_loader))

In [ ]:
# show all 3 channels of image 10
print(batch_images[0].shape)

fig = plt.figure(figsize=(5, 5))
plt.imshow(batch_images[11][0,:,:])
fig = plt.figure(figsize=(5, 5))
plt.imshow(batch_images[11][1,:,:])
fig = plt.figure(figsize=(5, 5))
plt.imshow(batch_images[11][2,:,:])

### different color maps
### cmap='bone', cmap = 'summer', cmap = 'seismic'

In [ ]:
# get the total amount of images in the dataset
num_train = len(train_dataset)

# create a list of indices for the whole dataset
indices = list(range(num_train))

# get the class labels from the dataset object (0-6)
class_labels = train_dataset.label
classes = [0, 1, 2, 3, 4, 5, 6]
# define the percentage of data that is not used for training
split_size = 0.2

# Define a Convolutional Neural Network

Pytorch makes it very easy to define a neural network. We have layers like Convolutions, ReLU non-linearity, Maxpooling etc. directly from torch library.

In this tutorial, we use The LeNet architecture introduced by LeCun et al. in their 1998 paper, [Gradient-Based Learning Applied to Document Recognition](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf). As the name of the paper suggests, the authors’ implementation of LeNet was used primarily for OCR and character recognition in documents.

The LeNet architecture is straightforward and small, (in terms of memory footprint), making it perfect for teaching the basics of CNNs.

In [ ]:
from torch import nn
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = len(classes)
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, (5,5), padding=2)
        self.conv2 = nn.Conv2d(6, 16, (5,5)) 
        self.fc1   = nn.Linear(16*5*5, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, num_classes)
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


## Task 1
Create a neural network capable of processing the image tensor 'img', which has only one channel. The network should contain at least one convolutional layer and one additional fully connected layer. Pay attention that within the fully connected layer the output dimension of the last convolutional layer has to fit the input dimension. Additionally, the output dimension of the fully connected layer before 'fc_fin' has to match the required input dimension of 'fc_fin'.

In [ ]:
img = torch.rand((1, 1, 200, 200))

output_dim = 6

class LeNet2(nn.Module):
    def __init__(self):
        super(LeNet2, self).__init__()
        # --------------------
        ########## Insert your layers here ##########
        self.conv1 = nn.Conv2d(1, 6, (5,5), padding=2)
        self.conv2 = nn.Conv2d(6, 16, (5,5)) 
        self.fc1   = nn.Linear(16*48*48, 120)
        self.fc2   = nn.Linear(120, 64)

        # --------------------
        self.fc_fin = nn.Linear(64, output_dim)

    def forward(self, x):
        # --------------------
        ########## Insert your forward pass here ##########
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # --------------------
        x = self.fc_fin(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

model = LeNet2()

task_done = False
output = model(img)

if output.size(1) == 6:
  print('The correct output size has been generated.')
  task_done = True


When your model processes the image correctly, submit your result by running the following cell:

In [ ]:
if task_done:
  write_result(1, str(model))
else:
  print("You didnt solve the task yet")

# Define a Loss function

Let's use a Classification Cross-Entropy loss.

$H_{y'} (y) := - \sum_{i} y_{i}' \log (y_i)$

### Median Frequency Balancing
There are datasets which have a large imbalance in the amount of label occurrence. A prediction would be therefore biased towards stronger represented classes. As a solution, we use **Median Frequency Balancing**.

In [ ]:
# Median Frequency Balancing

# get the class labels of each image
class_labels = train_dataset.label
# empty array for counting instance of each class
count_labels = np.zeros(len(classes))
# empty array for weights of each class
class_weights = np.zeros(len(classes))

# populate the count array
for l in class_labels:
  count_labels[l] += 1

# get median count
median_freq = np.median(count_labels)

# calculate the weigths
for i in range(len(classes)):
  class_weights[i] = median_freq/count_labels[i]

# print the weights
for i in range(len(classes)):
    print(classes[i],":", class_weights[i])

Now we define the loss function with the weights

In [ ]:
class_weights = torch.FloatTensor(class_weights).to(device)
criterion = nn.CrossEntropyLoss(weight = class_weights)


# Define the Optimizer

The most common and effective Optimizer currently used is **Adam: Adaptive Moments**. You can look [here](https://arxiv.org/abs/1412.6980) for more information.


In [ ]:
import torch.optim as optim

# now lets go back to the initial LeNet architecture
net = LeNet()
net = net.to(device)

# and define an optimizer
optimizer = optim.Adam(net.parameters(), lr=1e-5)
print(net)


# Training

After everything has been set up, we can now start an actual training on our MNIST dataset. To save time, for the moment we will run only ten epochs. Within the training, our dataloader is used to load a batch from our MNIST dataset. This batch is forwarded to the model. The corresponding output is compared against its labels with the chosen loss function, here called 'criterion'. Then, the loss values are backpropagated through the whole model.

In [ ]:
num_epochs = 5

def run_epoch():
  running_loss = 0.0
  data_loader = train_loader
  for i, data in enumerate(data_loader):
      # get the inputs
      inputs, labels = data
      inputs, labels = inputs.to(device), torch.argmax(labels, 1).to(device)
      
      # set the parameter gradients to zero
      optimizer.zero_grad()

      # forward + backward + optimize
      outputs = net(inputs[:,0:1,:,:])
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      
      #compute accuracy
      _, predicted = torch.max(outputs, 1)
      running_loss += loss.item()
  return running_loss

for epoch in range(num_epochs):
    
    running_loss = run_epoch()

    running_loss /= len(train_loader)

    print('Epoch: {}'.format(epoch+1))
    print('Loss: {}' .format(running_loss))

print('Finished Training')

## Task 2
Rewrite the 'run_epoch' function, so that it can perform the training and validation in consecutive steps for each epoch. By using the corresponding dataloaders, the data can be easily provided to the model. The train() and eval() function change the model into the corresponding state. Please note that by running the previous code box the model is now already trained. We therefore re-initialize the model.

In [ ]:
net = LeNet()
net = net.to(device)

optimizer = optim.Adam(net.parameters(), lr=1e-5)

num_epochs = 10

def run_epoch():
  loss_dict = {'train': 0.0,
               'val': 0.0}

  ########## Specifiy which dataloader needs to be used in a phase within a dictionary ##########
  data_loader = {'train': train_loader,
                  'val': val_loader}

  ########## ----------------------- ##########

  for phase in ['train', 'val']:

    running_loss = 0.0
    num_correct = 0
    num_all = 0

    for i, data in enumerate(data_loader[phase]):

      ########## get the inputs ##########
      inputs, labels = data
      inputs, labels = inputs.to(device), torch.argmax(labels, 1).to(device)
      ########## ------------------------------- ##########

      # set the parameter gradients to zero
      optimizer.zero_grad()

      if phase == 'train':
        net.train()  # Set model to training mode
      else:
        net.eval()  # Set model to validation mode

      # Calculate the network output and its loss 
      outputs = net(inputs[:,0:1,:,:])
      loss = criterion(outputs, labels)

      # The loss backpropagation and optimization step is only necessary when
      # in training mode:
      # -------------------------------
      ########## Implement the loss backpropagation for the training phase only ##########
      if phase == 'train':
        loss.backward()
        optimizer.step()

      ########## ------------------------------- ##########
      
      #compute accuracy
      _, predicted = torch.max(outputs, 1)

      running_loss += loss.item()
      if phase == 'val':
        num_correct += torch.sum(predicted == labels).item()
        num_all += labels.size()[0]

    # ----------------------------------
    ########## Save the correct loss for every phase within the dictionary ##########
    loss_dict[phase] = running_loss / len(data_loader[phase])
    ########## ------------------------------- ##########
  accuracy = num_correct/num_all
  return loss_dict, accuracy

run_through = False
for epoch in range(num_epochs):
    
    loss_dict, accuracy = run_epoch()

    print('Epoch: {}'.format(epoch+1))
    for phase in ['train', 'val']:
      print('Loss {}: {}' .format(phase, loss_dict[phase]))
      if phase == 'val':
        print('Validation Accuracy: {}%' .format(np.round(accuracy, 3)*100))
    print()
    run_through = True

print('Finished Training')

After you have sucessfully performed the training, please submit your results:

In [ ]:
if run_through:
  write_result(2, 'The last obtained validation loss is {}'.format(loss_dict['val']))
else:
  print("You didnt solve the task yet")

The following function is a useful tool to get information about your model:

In [ ]:
from torchsummary import summary
summary(net, input_size=(1, 28, 28))

# Homework
After performing the training and validation of your system you are now ready to perform the inference on your test set. Implement the inference step for out dataset:

In [ ]:
running_loss = 0.0
num_correct = 0
num_all = 0
data_loader = test_loader
for i, data in enumerate(data_loader):

  ########## Implement the data loading and prediction on the test set ##########
  
  # your code here

  ########## ------------------------------- ##########
  _, predicted = torch.max(outputs, 1)
  num_correct += torch.sum(predicted == torch.argmax(labels, 1)).item()
  num_all += labels.size()[0]
  running_loss += loss.item()
running_loss /= len(data_loader)

print('Loss for test set is {}'.format(running_loss))
print('Test accuracy of the network: {}%'.format(np.round(num_correct/num_all*100, 3)))


After you completed the inference, submit your test loss result here:

In [ ]:
if not running_loss == 0.0:
  write_result(3, 'The obtained test loss is {}'.format(running_loss))
else:
  print("You didnt solve the task yet")